## Exercise 1 (CIFAR Dataset)

* In this notebook I am re-using the same code as done for MNIST dataset there are only some changes in LeNet function since our dataset is changed as CIFAR has RGB data so we need to update the conv. layers accordingly. Other than that I made changes to pooling method in forward function. I tried to run this configuration with different learn rate but got a better result with lr = 0.001. In case of other learn rates I was getting a straight line after some point.
* Tensorboard report is not getting converted when pdf is generated, I have added the screenshots of some plots but I have also attached the log files folder which can be used to view the plots and images on a tensorboard running instance.

In [1]:
import numpy as np
from PIL import Image
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
import torch as pt 
from torch.utils.data import Dataset, DataLoader , Subset, ConcatDataset, ChainDataset
from torch.nn import functional as F
from torch import nn
from torchvision.datasets import MNIST, CIFAR10
from torchvision.transforms import ToTensor
from torch.utils.tensorboard import SummaryWriter

In [2]:
%load_ext tensorboard

tr_data = CIFAR10(root = ".", train = True, download = True, transform = ToTensor())
ts_data = CIFAR10(root = ".", train = False, download = True, transform = ToTensor())


Files already downloaded and verified
Files already downloaded and verified


In [3]:
class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, (3, 3)) 
        self.conv2 = nn.Conv2d(6, 16, (7, 7), padding = 1)
        self.fc1   = nn.Linear(16 * 5 * 5, 120)
        self.fc2   = nn.Linear(120, 84)
        self.fc3   = nn.Linear(84, 10)
    
    def forward(self, x):
        x = F.max_pool2d(pt.sigmoid(self.conv1(x)), (2, 2))
        x = F.max_pool2d(pt.sigmoid(self.conv2(x)), (2, 2))
        x = x.view(-1, 16 * 5 * 5)
        x = pt.sigmoid(self.fc1(x))
        x = pt.sigmoid(self.fc2(x))
        x = F.softmax(self.fc3(x),dim = 1)
        return x

In [4]:
b_size = 128

logs = 'runs/experiment_1_cifar'
sum_output = SummaryWriter(logs)

In [5]:
def lenet_tr(dataLoad, lenet_model, loss_func, adam_opt):
    dataSize = len(dataLoad.dataset)
    avg_loss, accu = 0, 0
    for batch, (X, y) in tqdm(enumerate(dataLoad), leave = False):
        X, y = X.to('cpu'), y.to('cpu')
        pred = lenet_model(X)
        loss_val = loss_func(pred, y)
        adam_opt.zero_grad()
        loss_val.backward()
        adam_opt.step()
        accu = accu + (pred.argmax(1) == y).type(pt.float).sum().item()
        avg_loss = avg_loss + loss_val.item()

    accu = accu / dataSize  
    avg_loss = avg_loss / (dataSize / len(X))
    print("Training accuracy: ", str(100 * accu), " Average loss: ", str(avg_loss))
    return avg_loss, (100 * accu)


def lenet_ts(dataLoad, lenet_model, loss_func):
    dataSize = len(dataLoad.dataset)
    ts_loss, accu = 0, 0

    with pt.no_grad():
        for X, y in tqdm(dataLoad, leave = False):
            X, y = X.to('cpu'), y.to('cpu')
            pred = lenet_model(X)
            ts_loss = ts_loss + loss_func(pred, y).item()
            accu = accu + (pred.argmax(1) == y).type(pt.float).sum().item()

    ts_loss = ts_loss / (dataSize / len(X))
    accu = accu / dataSize
    print("Testing Accuracy: ", str(100 * accu), " Average loss: ", (ts_loss))
    return ts_loss, (100 * accu)

In [6]:
lr = 1e-3
tr_dl = DataLoader(tr_data, batch_size = b_size, shuffle = True)
ts_dl = DataLoader(ts_data, batch_size = b_size, shuffle = False) # True -> False

lenet_model = LeNet()
lenet_model.to('cpu')
loss_func = nn.CrossEntropyLoss()

adam_opt = pt.optim.Adam(lenet_model.parameters(), lr = lr, betas = (0.9, 0.999), eps = 1e-08, weight_decay = 0, amsgrad = False)

epochs = 20
for i in tqdm(range(epochs)):
    print("Epoch: ", (i+1))
    tr_loss, tr_accu = lenet_tr(tr_dl, lenet_model, loss_func, adam_opt)
    ts_loss, ts_accu = lenet_ts(ts_dl, lenet_model, loss_func)    
    if sum_output is not None:
        sum_output.add_scalars(f'Loss with lr = {lr}', {"Train" : tr_loss,
                                     "Test" : ts_loss}, i)
        sum_output.add_scalars(f'Accuracy with lr = {lr}', {"Train" : tr_accu,
                                          "Test" : ts_accu}, i)


  0%|          | 0/20 [00:00<?, ?it/s]

Epoch:  1


0it [00:00, ?it/s]

Training accuracy:  15.098  Average loss:  1.4224973484039307


  0%|          | 0/79 [00:00<?, ?it/s]

...

Testing Accuracy:  43.47  Average loss:  0.2561337312698364
Epoch:  20


0it [00:00, ?it/s]

Training accuracy:  43.309999999999995  Average loss:  1.267886409187317


  0%|          | 0/79 [00:00<?, ?it/s]

Testing Accuracy:  43.56  Average loss:  0.2558630283355713


In [7]:
a_sample = iter(tr_dl).next()[0][np.random.randint(b_size)]

activList = dict()
def get_activation(name):
    def hook(lenet_model, input, output):
        activList[name] = output.detach()
    return hook


lenet_model.conv1.register_forward_hook(get_activation('conv1'))
lenet_model.conv2.register_forward_hook(get_activation('conv2'))
lenet_model.cpu()

sum_output.add_image(f'Original Image', a_sample)

output = lenet_model(a_sample.unsqueeze(0))

active_1st = activList['conv1'].squeeze()
active_2nd = activList['conv2'].squeeze()

filtrs = active_1st.size(0)
fig, axes = plt.subplots(1, filtrs, figsize = (filtrs * 2, 2))
for i in range(filtrs):
    axes[i].imshow(active_1st[i].numpy())
    axes[i].axis('off')

sum_output.add_figure('1st Conv. layer', fig)
plt.close()

filtrs = active_2nd.size(0)
fig, axes = plt.subplots(4, filtrs // 4, figsize = (filtrs // 2, 8))
for i in range(filtrs):
    x,y = i%4,i//4 
    axes[x,y].imshow(active_2nd[i].numpy())
    axes[x,y].axis('off')

sum_output.add_figure('2nd Conv. layer', fig)
plt.close()

listKernel = lenet_model.conv1.weight.detach()
filtrs = listKernel.size(0)
fig, axes = plt.subplots(1, filtrs, figsize = (filtrs * 2, 2))
for idx in range(filtrs):
    axes[idx].imshow(listKernel[idx].squeeze())
    axes[idx].axis('off')

sum_output.add_figure('1st Conv. feature-map', fig)
plt.close()

listKernel = lenet_model.conv2.weight.detach()
fig, axes = plt.subplots(listKernel.size(1), listKernel.size(0), figsize = (listKernel.size(0), listKernel.size(1)))
for i in range(listKernel.size(0)):
    for j in range(listKernel.size(1)):
        axes[j, i].imshow(listKernel[i,j].squeeze())
        axes[j, i].axis('off')

sum_output.add_figure('2nd Conv. feature-map', fig)
plt.close()
sum_output.close()


Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


In [9]:
%tensorboard --logdir runs/experiment_1_cifar --host localhost --port 8094